In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import time
from zipfile import ZipFile
import warnings
import pickle
import torch
from torch.utils.data import Dataset
import tsfel
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
import lightgbm as lgb
from hiclass import LocalClassifierPerNode, LocalClassifierPerParentNode, LocalClassifierPerLevel
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
warnings.filterwarnings('ignore')

In [21]:
base_dir = "../logs/ensemble/base_ensemble/01_27_2025-12_43_22"

In [22]:
os.listdir(base_dir)

['xgb', 'rf', 'lgb']

In [ ]:
"""
TODO:
1. 
"""

In [23]:
cv_res = pd.read_csv(os.path.join(base_dir, 'rf/cv_report.csv'))
cv_res

,col,precision,recall,f1,support
0,Active_Power_Sensor,0.174897,0.273752,0.213434,NaN
1,Air_Flow_Sensor,0.182297,0.272370,0.218411,NaN
2,Air_Flow_Setpoint,0.098485,0.144444,0.117117,NaN
3,Air_Temperature_Sensor,0.264496,0.298428,0.280440,NaN
4,Air_Temperature_Setpoint,0.232540,0.277473,0.253027,NaN
...,...,...,...,...,...
90,Zone_Air_Dewpoint_Sensor,0.301282,0.261111,0.279762,NaN
91,Zone_Air_Humidity_Sensor,0.318627,0.300926,0.309524,NaN
92,Zone_Air_Humidity_Setpoint,0.000000,0.000000,0.000000,NaN
93,Zone_Air_Temperature_Sensor,0.289678,0.294362,0.292001,NaN


In [27]:
avg = [
    pd.read_csv(os.path.join(base_dir, "rf/test_predictions", f"final_result_{i}.csv"))
    for i in range(2)
]


In [29]:
import pandas as pd

def merge_mean(df_list):
    """
    Given a list of DataFrames with identical columns and index,
    return one DataFrame whose numeric columns are the mean of all
    those numeric columns across the list.

    The "filename" column is assumed to be the same or can be taken
    from the first DataFrame.
    """
    # Make a copy of the first DataFrame to initialize the output
    merged_df = df_list[0].copy()

    # Identify all numeric columns (excluding "filename")
    numeric_cols = merged_df.columns.drop("filename")

    # Sum up the numeric columns from all other DataFrames
    for df in df_list[1:]:
        merged_df[numeric_cols] += df[numeric_cols]

    # Divide by the number of DataFrames to get the mean
    merged_df[numeric_cols] /= len(df_list)

    return merged_df


# Example usage:
# Suppose dfs is the list of DataFrames you showed in the question
# final_df = merge_mean(dfs)
# print(final_df)


In [32]:
def check_pred_num(_final_res, thr=0.4):
    # Exclude 'filename' column if it exists
    filtered_df = _final_res.drop(columns=['filename'], errors='ignore')

    return (filtered_df >= thr).sum(axis=1)

In [33]:
m_res = merge_mean(avg)

In [35]:
check_pred_num(m_res).value_counts()

2     85307
3     74616
4     58612
1     52155
5     31619
6      9940
7      2610
8       726
9       119
10       16
Name: count, dtype: int64